In [1]:
import tiktoken

In [3]:
with open("C:\\Users\\Admin\\Downloads\\the-verdict.txt",encoding='utf-8') as f:
    text = f.read()

print(len(text))

20479


In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')

In [6]:
enc_text = tokenizer.encode(text)
print(len(enc_text))

5145


In [7]:
enc_sample = enc_text[50:]

In [10]:
context_size = 4

for i in range(1,context_size+1):
    x = enc_sample[:i]
    y = enc_sample[i]

    print(tokenizer.decode(x),' > ',tokenizer.decode([y]))

 and  >   established
 and established  >   himself
 and established himself  >   in
 and established himself in  >   a


In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [12]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
import torch

dataloader = create_dataloader_v1(
    text,batch_size=1,max_length=4,stride=1,shuffle=False   
)